# Description of this file
In this file the data for the project is processed into a more suitable form (simple table) since the original data format is not ready for analysis. So, a type of Extraction from the ETL process is performed.

## Quicklinks for this file
1. [Import of packages](#Packages-import)
2. [Class declaration of the extractor for cuadros](#Class-declaration-of-the-extractor-for-cuadros)

## Packages-import

In [215]:
# Data extraction
import openpyxl as opx

# Data manipulation
import pandas as pd

## Class-declaration-of-the-extractor-for-cuadros

In [216]:
"""
I need: which table to extract, there are three in every cuadro, so maybe that could be an enum.
I have to locate the table in the sheet, i.e: cell where it starts
Extract the titles of cols (Melt year and quarter into one column)
Go row by row and fatten based on section while adding to gdp_per_concept dataframe. (Keep the title of each section in a separate dataframe)
Once you get to gdp summary, add the data to gdp_summary dataframe.
"""

class Table:
    def __init__(self):
        self.gdp_summary = pd.DataFrame()
        self.gdp_per_concept = pd.DataFrame()
        self.worksheet = None
        self.titles = []
        self.__Concepto_title_idx = 0
        self.first_cell_coord = ""

    def extract(self, path: str, cuadro: int, first_cell: str):
        self.worksheet = opx.load_workbook(path)["Cuadro " + str(cuadro)]
        self.first_cell_coord = first_cell

        self._extract_titles()
        self._extract_data()

    def _extract_titles(self):
        at_end_of_cols = False

        first_cell = self.worksheet[self.first_cell_coord]

        col_offset_to_firstcell = 0
        while not at_end_of_cols:

            title_temp = first_cell.offset(0, col_offset_to_firstcell).value
            sub_title = first_cell.offset(1, col_offset_to_firstcell).value

            if title_temp is not None: # Safe guard to avoid "None" in the title when melting year and quarter titles
                title = str(title_temp).encode("ascii", "ignore").decode("ascii")
            elif sub_title is None: # If both title and subtitle are None, we are at the end of the table
                at_end_of_cols = True
                break

            if (sub_title is not None):
                # There is only sub_title for titles with years, so we can safely take the first 4 characters... unlesss it is the year 10000+, in which case... hi future!
                self.titles.append(f"{title[:4]} {sub_title}")
            else:
                # This is saved to identify where this col is located and ignore in the gdp_per_concept dataframe
                if title == "Concepto":
                    self.__Concepto_title_idx = col_offset_to_firstcell
                self.titles.append(title)

            col_offset_to_firstcell += 1
        
        self.gdp_per_concept = pd.DataFrame(columns=self.titles)
        self.gdp_summary = pd.DataFrame(columns=self.titles)
                
    def _extract_data(self):
        at_end_of_cols = False
        at_end_of_rows = False

        first_cell = self.worksheet[self.first_cell_coord].offset(3, 0) # One row is merged with the cell at first_cell_coord and another is empty

        col_offset_to_firstcell = 0
        row_offset_to_firstcell = 0 # The first row is the titles, the second is empty

        table_rows = []

        while not at_end_of_rows:

            row_data = []

            while not at_end_of_cols:
                current_cell = first_cell.offset(row_offset_to_firstcell, col_offset_to_firstcell)
                current_cell_value = current_cell.value

                # Check we're still inside the table
                print(current_cell_value)
                if current_cell.offset(-1, 0).value is not None \
                    and current_cell_value is None \
                    and current_cell.offset(1, 0).value is not None \
                    and col_offset_to_firstcell == 0:

                    at_end_of_rows = True
                    break
                if (current_cell_value is None) and current_cell.offset(0, 1).value is None and current_cell.offset(0, 2).value is None:

                    col_offset_to_firstcell = 0
                    at_end_of_cols = True
                    break

                # Save the value
                if col_offset_to_firstcell != self.__Concepto_title_idx and col_offset_to_firstcell != 0:
                    row_data.append(current_cell_value)

                col_offset_to_firstcell += 1

            table_rows.append(row_data)

            at_end_of_cols = False
            row_offset_to_firstcell += 1

        self.gdp_per_concept = pd.DataFrame(table_rows, columns=[title for title in self.titles[1:] if title != self.titles[self.__Concepto_title_idx]])
        self.gdp_summary = self.gdp_per_concept.tail(4)

        print(self.gdp_per_concept.info())
        print(self.gdp_summary.info())


In [217]:
data_file_path = "../data/Anexos_produccion_constantes_IV_2022.xlsx"

table = Table()
table.extract(data_file_path, 3, "A12")
table.titles


None
A
None
Agricultura, ganadería, caza, silvicultura y pesca
8877.787199308521
9244.622655808418
10038.738190863995
9747.851954019066
9010.87328329429
9224.677705026977
10410.929633699592
10070.519377979143
9374.099323290202
9564.373581033899
10920.28515555663
10380.241940119271
9586.498502134029
9584.28356676827
10802.161549969214
9942.056381128481
9294.735492137122
9322.631270578573
11007.121409569407
10197.5118277149
9267.576817060446
9542.604961242372
10720.04510025604
10412.773121441138
9893.20995707664
9713.867063468904
10776.530416192205
10322.392563262249
10063.578033314754
10224.247152875301
11039.16730923613
10398.007504573816
10410.50343110908
11372.653958894358
11842.27246893334
11209.570141063221
11209.034099014518
11280.477364922042
12056.807485437072
11593.681050626372
11560.808085551922
11588.563601399674
12893.801971694946
12080.826341353459
11491.731539646722
11705.376487214293
13090.25082628463
13153.641146854356
12783.374877405413
12444.581464155666
13879.32902453

['Clasificacin Cuentas Nacionales',
 'Secciones y divisiones CIIU Rev. 4 A.C.\n25 agrupaciones',
 'Divisiones CIIU Rev. 4 A.C.\n61 agrupaciones',
 'Concepto',
 '2005 I',
 '2005 II',
 '2005 III',
 '2005 IV',
 '2006 I',
 '2006 II',
 '2006 III',
 '2006 IV',
 '2007 I',
 '2007 II',
 '2007 III',
 '2007 IV',
 '2008 I',
 '2008 II',
 '2008 III',
 '2008 IV',
 '2009 I',
 '2009 II',
 '2009 III',
 '2009 IV',
 '2010 I',
 '2010 II',
 '2010 III',
 '2010 IV',
 '2011 I',
 '2011 II',
 '2011 III',
 '2011 IV',
 '2012 I',
 '2012 II',
 '2012 III',
 '2012 IV',
 '2013 I',
 '2013 II',
 '2013 III',
 '2013 IV',
 '2014 I',
 '2014 II',
 '2014 III',
 '2014 IV',
 '2015 I',
 '2015 II',
 '2015 III',
 '2015 IV',
 '2016 I',
 '2016 II',
 '2016 III',
 '2016 IV',
 '2017 I',
 '2017 II',
 '2017 III',
 '2017 IV',
 '2018 I',
 '2018 II',
 '2018 III',
 '2018 IV',
 '2019 I',
 '2019 II',
 '2019 III',
 '2019 IV',
 '2020 I',
 '2020 II',
 '2020 III',
 '2020 IV',
 '2021 I',
 '2021 II',
 '2021 III',
 '2021 IV',
 '2022 I',
 '2022 II',
 '

In [218]:
print(sheet1)

NameError: name 'sheet1' is not defined